In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras import layers

In [4]:
dados = "O Yan vai tirar 10 na avaliação, pois ele conseguiu colar na prova"
frase_dividida = dados.split()

tokenizer = Tokenizer(oov_token ='<unk>')
tokenizer.fit_on_texts([frase_dividida])
indices_palavras = tokenizer.word_index
vocab_tamanho =  len(indices_palavras)+1


In [6]:
entradas = []
saida = []

for i in range(1, len(frase_dividida)):
    sequencia = frase_dividida[:i+1]
    entradas.append(sequencia[:-1])
    saida.append(sequencia[-1])

sequencias_entradas = tokenizer.texts_to_sequences(entradas)
sequencias_saidas = tokenizer.texts_to_sequences(saida)



In [7]:
max_tamanho_seq = max([len(x) for x in sequencias_entradas])
sequencias_entradas = tf.keras.preprocessing.sequence.pad_sequences(sequencias_entradas,maxlen = max_tamanho_seq, padding = 'pre')

In [8]:
modelo = Sequential()
modelo.add(Embedding(input_dim = vocab_tamanho, output_dim = 10, input_length = max_tamanho_seq))

modelo.add(LSTM(100, return_sequences = True))
modelo.add(Dropout(0.3))

modelo.add(LSTM(100))
modelo.add(Dropout(0.3))

modelo.add(Dense(50, activation = 'relu'))

modelo.add(Dense(vocab_tamanho, activation = 'softmax'))


C:\Users\mathe\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
modelo.compile(optimizer = 'nadam', loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])
modelo.fit(sequencias_entradas, np.array(sequencias_saidas), epochs = 400, verbose = 0)

In [13]:
def prever_proxima_palavra(modelo, tokenizer, sequencia_entrada,max_tamanho):
    sequencia_num = tokenizer.texts_to_sequences([sequencia_entrada])
    sequencia_num = tf.keras.preprocessing.sequence.pad_sequences(sequencia_num, maxlen=max_tamanho, padding='pre')
    
    predicao = modelo.predict(sequencia_num)
    indice_predito = np.argmax(predicao, axis=-1)
    
    indice_para_palavra = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
    
    return indice_para_palavra.get(indice_predito[0], '<unk>')

In [15]:
frase_teste = "O Yan vai tirar 10 na avaliação, pois ele conseguiu "
palavra_predita = prever_proxima_palavra(modelo, tokenizer, frase_teste, max_tamanho_seq)
print(f"\nFrase de entrada: '{frase_teste}'")
print(f"Próxima palavra prevista: '{palavra_predita}'")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step

Frase de entrada: 'O Yan vai tirar 10 na avaliação, pois ele conseguiu '
Próxima palavra prevista: 'colar'
